In [1]:
import numpy as np
import pandas as pd
import re
file_path = "../Data/properatti.csv"
#pd.set_option('display.max_row', 121300)
pd.set_option('display.max_columns', 500)
df = pd.read_csv(file_path, index_col=0)

In [2]:
#Hace un conteo de los valores nulos por columnas
df.isnull().sum()

operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

In [3]:
#va a la columna 'place_with_parent_names' y va separando los conceptos por '|'
df['country']=df["place_with_parent_names"].str.split('|').str[1]
df['state']=df["place_with_parent_names"].str.split('|').str[2]
df['place']=df["place_with_parent_names"].str.split('|').str[3]
df['extra_information']=df["place_with_parent_names"].str.split('|').str[4] #En caso que sea el nombre de un barrio privado

In [4]:
#Ordena el dataframe
df = df.sort_values(["country", "state","place","extra_information","property_type"])

In [5]:
#Recorre el dataframe, busca aquellas publicaciones que tengan menor superficie total que cubierta y lo elimina
for index, x in df.iterrows():
       if x['surface_total_in_m2'] < x['surface_covered_in_m2']:
            df=df.drop([index])

In [6]:
df['regex_titulo'] = df['title'].str.extract('(U[$S]D ?\d+[\.\s]?\d+[\.\s]?\d+)', expand=False)
#df[df['regex_titulo'].notnull()]

In [7]:
df['regex_titulo']=df['regex_titulo'].str.extract('(\d+[\.\s]?\d+[\.\s]?\d+)', expand=False)
#df[df['regex_titulo'].notnull()]

In [8]:
df['regex_desc'] = df['description'].str.extract('(U[$S]D ?\d+[\.\s]?\d+[\.\s]?\d+)', expand=False)
#df[df['regex_desc'].notnull()]

In [9]:
df['regex_desc']=df['regex_desc'].str.extract('(\d+[\.\s]?\d+[\.\s]?\d+)', expand=False)
#df[df['regex_desc'].notnull()]

In [10]:
df.loc[22646,'regex_desc']=np.nan
df.loc[2370,'regex_desc']=np.nan
df.loc[4013,'regex_desc']=np.nan
df.loc[52624,'regex_desc']=np.nan
df.loc[3418,'regex_desc']=np.nan
df.loc[3420,'regex_desc']=np.nan
df.loc[3421,'regex_desc']=np.nan
df.loc[3422,'regex_desc']=np.nan
df.loc[3423,'regex_desc']=np.nan
df.loc[3424,'regex_desc']=np.nan
df.loc[3425,'regex_desc']=np.nan
df.loc[3426,'regex_desc']=np.nan
df.loc[3427,'regex_desc']=np.nan
df.loc[3428,'regex_desc']=np.nan
df.loc[3429,'regex_desc']=np.nan
df.loc[3430,'regex_desc']=np.nan
df.loc[3431,'regex_desc']=np.nan
df.loc[3432,'regex_desc']=np.nan
df.loc[3433,'regex_desc']=np.nan
df.loc[3434,'regex_desc']=np.nan
df.loc[3435,'regex_desc']=np.nan
df.loc[3436,'regex_desc']=np.nan
df.loc[3437,'regex_desc']=np.nan
df.loc[3438,'regex_desc']=np.nan
df.loc[3439,'regex_desc']=np.nan
df.loc[3440,'regex_desc']=np.nan
df.loc[3441,'regex_desc']=np.nan
df.loc[3442,'regex_desc']=np.nan
df.loc[3443,'regex_desc']=np.nan
df.loc[3444,'regex_desc']=np.nan
df.loc[41543,'regex_desc']=np.nan
df.loc[22995,'regex_desc']=np.nan
df.loc[1726,'regex_desc']=np.nan

df.loc[22663,'regex_titulo']=np.nan
df.loc[90556,'regex_titulo']=np.nan

df=df.drop([2791])
df=df.drop([3897])

for index, x in df.iterrows():
        if x['regex_titulo'] == x['regex_desc']:
             df.loc[index,'regex_desc']=np.nan

In [11]:
df.regex_titulo.count(),df.regex_desc.count()#2482,5450

(2482, 5450)

In [ ]:
df[(df.price.isnull())&(df.regex_titulo.isnull())&(df.regex_desc.isnull())]
#17797 precio,regex nulos

In [17]:
 for index, x in df.iterrows():
        if  pd.notnull(x.regex_titulo):
            df.loc[index,'regex_price']= x['regex_titulo']
            df.loc[index,'regex_titulo']=np.nan
        elif pd.notnull(x.regex_desc):
            df.loc[index,'regex_price']= x['regex_desc']
            df.loc[index,'regex_desc']=np.nan

In [22]:
df=df.drop(['regex_titulo', 'regex_desc'], axis=1)

In [47]:
# df[(df.price.notnull())&(df.regex_price.notnull())&(df.price!=df.regex_price)]
for index, x in df.iterrows():
    if (pd.notnull(x.price)) & (pd.notnull(x.regex_price)):
        df.loc[index,'regex_price']= np.nan

In [48]:
len(df[(df.price.isnull())&(df.regex_price.isnull())])

17797

In [50]:
for index, x in df.iterrows():
    if (pd.isnull(x.price)) & (pd.notnull(x.regex_price)):
        df.loc[index,'price']= x['regex_price']
        df.loc[index,'regex_price']= np.nan
        df.loc[index,'currency']= 'USD'
df=df.drop(['regex_price'], axis=1)

17797

In [ ]:
#cantidad de regex_titulo y regex_desc no nulos a la vez / 108
#cantidad de regex_titulo y regex_desc distintos entre si / 37
#df[(df.regex_titulo.notnull())&(df.regex_desc.notnull())&(df.regex_titulo==df.regex_desc)]
#df[(df.regex_titulo.notnull())&(df.regex_desc.notnull())]
#df.loc[2791,'regex_desc']=np.nan
#df=df.drop([3897])

In [ ]:
#df.pivot_table(index=["country","state","place"], values=['price','price_aprox_local_currency','price_aprox_usd','price_per_m2','price_usd_per_m2','surface_total_in_m2','surface_covered_in_m2'])
#df.pivot_table(index=["country","state"],columns=['property_type'], values=['price'])
#df[(df.regex_titulo.notnull())&(df.regex_desc.notnull())&(df.regex_titulo!=df.regex_desc)]


In [ ]:
#df['precioReal']=df.apply(lambda x: (x.price/17.6) if x.currency=='ARS' else(x.price),axis=1)